In [1]:
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [2]:
from case_code.code_downloader import CodeDownloader, get_metadata
from case_code.code_loader import CodeLoader
from case_code.code_transformer import transform

# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()

In [3]:
metadata = get_metadata()
print(f"{'Total files downloaded:':<24} {metadata['total_downloads']}")
print(f"{'Total cases:':<24} {metadata['total_cases']}")

Total files downloaded:  125
Total cases:             110


In [4]:
import pandas as pd

# Transform the downloaded code to case
loader = CodeLoader()
transformed_data = await transform(loader=loader)

data = [(key.value, value) for key, value in transformed_data.items()]
df = pd.DataFrame(data, columns=["Title", "Count"])
print(df.to_string(index=False))

Start processing files...


Processing files: 100%|██████████| 125/125 [06:08<00:00,  2.95s/file]

                  Title  Count
            Total cases    109
Skipped: case not found     57
   Skipped: parse error      0


In [5]:
from case_code.code_transformer import get_transformation_result

df = get_transformation_result()
filtered_df = df.loc[df["transformable"] == False]
filtered_df

,id,transformable
41,bridge_eth_scroll_optimism,False


In [6]:
from case import get_retriever

retriever = get_retriever()

In [7]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 5, 'source': 'eigenpie/index.ts', 'steps': "[{'description': 'Stake ETH to Lido', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': 'unknown', 'function_name': 'submit', 'input_args': ['ReferalAccount']}, {'description': 'Approve stETH to Eigenpie', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '0', 'function_name': 'approve', 'input_args': ['0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'unknown']}, {'description': 'Stake stETH to Eigenpie', 'to': '0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'value': '0', 'function_name': 'depositAsset', 'input_args': ['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'unknown', 'unknown', 'ReferalAccount']}]", 'total_steps': 3}, page_content='Eigenpie staking for stETH on Lido and Eigenpie'),
 Document(metadata={'case_id': 'stake_eth', 'seq_num': 78, 'source': 'stake-eth/stake.ts', 'steps': "[{'description': 'Stake into Lido', 'to': 'unknown', 'value': 'unknown', 'funct

In [14]:
retriever.invoke("Swap 10 USDC to USDT")

[Document(metadata={'case_id': 'etf_base_meme_top10', 'seq_num': 109, 'source': 'etf-basememe-top10/index.ts', 'steps': "[{'description': 'Approve USDC for 1inch', 'to': 'oneInchAddress', 'value': 'unknown', 'function_name': 'approve', 'input_args': ['USDCAddr', '1inch']}, {'description': 'Swap USDC to Top1 token', 'to': 'oneInchAddress', 'value': 'unknown', 'function_name': 'swap', 'input_args': ['USDCAddr', 'Top1', 'amount']}, {'description': 'Swap USDC to Top2 token', 'to': 'oneInchAddress', 'value': 'unknown', 'function_name': 'swap', 'input_args': ['USDCAddr', 'Top2', 'amount']}, {'description': 'Swap USDC to Top3 token', 'to': 'oneInchAddress', 'value': 'unknown', 'function_name': 'swap', 'input_args': ['USDCAddr', 'Top3', 'amount']}, {'description': 'Swap USDC to Top4 token', 'to': 'oneInchAddress', 'value': 'unknown', 'function_name': 'swap', 'input_args': ['USDCAddr', 'Top4', 'amount']}, {'description': 'Swap USDC to Top5 token', 'to': 'oneInchAddress', 'value': 'unknown', 'fu

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from models import Case
from utils.model_selector import get_chat_model


template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()

model = get_chat_model()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [9]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_0_03_eth_lido_eigenpie", "description": "Stake 0.03 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake 0.03 ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0.03", "function_name": "submit", "input_args": ["ReferalAccount"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "unknown"]}, {"description": "Stake stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "unknown", "unknown", "ReferalAccount"]}]}


In [10]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_eth_lido_eigenpie", "description": "Stake 0.03 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake 0.03 ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0.03", "function_name": "submit", "input_args": ["ReferalAccount"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "unknown"]}, {"description": "Deposit stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "unknown", "unknown", "ReferalAccount"]}]}


In [11]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_usdt_for_usdc", "description": "Swap 100 USDT for USDC on Uniswap", "total_steps": 4, "steps": [{"description": "Approve USDT for Uniswap", "to": "0xdac17f958d2ee523a2206206994597c13d831ec7", "value": "0", "function_name": "approve", "input_args": ["0xe592427a0aece92de3ede1f18e0157c05861564", "100"]}, {"description": "Swap USDT to USDC", "to": "0xe592427a0aece92de3ede1f18e0157c05861564", "value": "0", "function_name": "exactInputSingle", "input_args": ["0xdac17f958d2ee523a2206206994597c13d831ec7", "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "100", "0xYourAddress", "0"]}, {"description": "Approve USDC for further use", "to": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "value": "0", "function_name": "approve", "input_args": ["0xYourNextAddress", "100"]}, {"description": "Transfer USDC to another address", "to": "0xYourNextAddress", "value": "100", "function_name": "transfer", "input_args": ["0xRecipientAddress", "100"]}]}


In [12]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_eth_for_usdc_uniswap", "description": "Swap 0.001 ETH for USDC on Uniswap", "total_steps": 1, "steps": [{"description": "Swap 0.001 ETH for USDC", "to": "UniswapAddress", "value": "0.001", "function_name": "swap", "input_args": ["ETH", "USDC", "0.001"]}]}


In [15]:
question = "Swap 20 USDC to USDT"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_usdc_to_usdt", "description": "Swap 20 USDC to USDT", "total_steps": 2, "steps": [{"description": "Approve USDC for 1inch", "to": "oneInchAddress", "value": "unknown", "function_name": "approve", "input_args": ["USDCAddr", "1inch"]}, {"description": "Swap USDC to USDT", "to": "oneInchAddress", "value": "unknown", "function_name": "swap", "input_args": ["USDCAddr", "USDT", "20"]}]}


In [13]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())

{"id": "yield_farming_multiple_protocols", "description": "Yield Farming with Multiple Protocols", "total_steps": 4, "steps": [{"description": "Deposit ETH into AAVE to earn interest.", "to": "AAVE", "value": "unknown", "function_name": "deposit", "input_args": ["ETH"]}, {"description": "Use aToken (aETH) as collateral to borrow DAI.", "to": "AAVE", "value": "unknown", "function_name": "borrow", "input_args": ["DAI", "aETH"]}, {"description": "Deposit the borrowed DAI into Yearn Finance.", "to": "Yearn Finance", "value": "unknown", "function_name": "deposit", "input_args": ["DAI"]}, {"description": "Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees.", "to": "Uniswap", "value": "unknown", "function_name": "addLiquidity", "input_args": ["Yearn vault tokens"]}]}
